In [1]:
!pip install arrow
from google.colab import auth, files, drive
import os
import pandas as pd
import arrow
import google.auth
from google.cloud import bigquery
from google.cloud import bigquery_storage
import itertools
import datetime

#Authentificating user.
auth.authenticate_user()
print('Authenticated')
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Authenticated
Mounted at /content/drive/


In [2]:
#Making directories for new incentive files.
today=arrow.now().format('MMDDYY')
today_path='QC- '+today

weekend_dir='/content/drive/My Drive/Incentive QCs/'+today_path+'/weekends'
weekday_dir='/content/drive/My Drive/Incentive QCs/'+today_path+'/weekdays'

os.makedirs('/content/drive/My Drive/Incentive QCs/'+today_path)
os.makedirs(weekend_dir)
os.makedirs(weekday_dir)

In [3]:
#Reading up the file with the weekly proposal.
os.chdir('/content/drive/MyDrive/')
incentive_columns=columns=list(pd.read_csv("/content/drive/MyDrive/QC-112320/incentives/11_20_20_FIS_CFA_v1.csv").columns)
incentive_columns=[x.lower() for x in incentive_columns]

df=pd.read_csv("proposal.csv")
df=df.dropna(how = 'all')
columns=list(df.columns[:-1])
df=df[columns]

#Reading up the file with places uuids for location-specific incentives.
mcd=pd.read_csv('/content/drive/MyDrive/incentives_creation/McD_uuids.csv')
sgf=pd.read_csv('/content/drive/MyDrive/incentives_creation/SGF_uuids.csv')
cfa=pd.read_csv('/content/drive/MyDrive/incentives_creation/CFA_uuids.csv')

In [4]:
#Reading the info about performance from last week (that we need for guaranteed earnings calculation), couriers' average earnings and georegions.

georegions="""SELECT DISTINCT 
case when name in ('Manhattan, New York City') then 'New York City'
  else name end as name
, id
, timezone
, market_id
,name as name2
FROM `datafall-prod.fleet_incentive.geo_regions`"""


credentials, your_project_id = google.auth.default(
    scopes=["https://www.googleapis.com/auth/cloud-platform"]
)

bqclient = bigquery.Client(credentials=credentials, project="datafall-fulfillment")
bqstorageclient = bigquery_storage.BigQueryReadClient(credentials=credentials)

georegions = bqclient.query(georegions).result().to_dataframe()
#Some states are written with comma and some without after city name, which prevents it from a correct joining.
geo=georegions
geo = geo.replace(',','', regex=True)

#Couriers average earnings.
avg_earnings = """select case when j.market in ('Dallas / Ft. Worth') then 'Dallas / Ft. Worth, Dallas' else j.market end, cm.id as market_id
     , avg(courier_price) as avg_courier_price
  from `postmates-reporting.stats.job` j
      left join `datafall-prod.postal_main.core_market` as cm
        on cm.name = j.market
 where complete is true
   and date(job_dt_local) >=date_sub(current_date(),interval 28 day)
   and date(job_dt_local) < date(current_timestamp())
     and j.fsm_state in ('DidCompleteDropoff','DidAdminCancel','DidCustomerCancel','DidDispatchCancel','DidDispatchRequestException')
    and j.fraud_canceled is false
    and j.charge_state != 'auth_failed'
    and j.is_pickup is false
    and j.is_return_job is false
 group by 1,2
 order by 1,2"""

avg_earnings = bqclient.query(avg_earnings).result().to_dataframe()

#4days.

days4 = """With l28_couriers as
  (select courier_uuid,
  market,
  job_id,
  job_dt_local
  from `postmates-reporting.stats.job` j
  where complete is true
  and date(job_dt_local) >=date_sub(current_date(),interval 28 day)
  and date(job_dt_local) < date(current_timestamp())
  and j.fsm_state in ('DidCompleteDropoff','DidAdminCancel','DidCustomerCancel','DidDispatchCancel','DidDispatchRequestException')
  and j.fraud_canceled is false
  and j.charge_state != 'auth_failed'
  and j.is_pickup is false
  and j.is_return_job is false)

---------------------------------------------------------------------------------------------
 ,peak_deliveries as
---------------------------------------------------------------------------------------------
  (select market, courier_uuid,
  case  
  when extract(dayofweek from date(job_dt_local)) in (5,6,7,1) and date(job_dt_local)>=date_sub(DATE_TRUNC(CURRENT_DATE(), WEEK(MONDAY)),interval 7 day) then '4day' end as tag,
  count(job_id) as jobs
  from l28_couriers
  group by 1,2,3
  order by 1,2,3
  
  ) 

select  distinct market, tag,  CONCAT(market, "_", tag) as concatenated,      PERCENTILE_DISC(jobs,0.97)
            OVER ( PARTITION BY market,tag ) AS PercentileDisc97
    FROM
(select * from peak_deliveries
 where tag is not null)


 order by 4 desc
 """
days4 = bqclient.query(days4).result().to_dataframe()

#Peak incentives.

peak="""with peak_deliveries_s1 as (
  
  select
      market
    , courier_uuid
    , date(job_dt_local) as dt
    , case

        -- Monday
        when extract(dayofweek from date(job_dt_local)) = 2 and time(job_dt_local) between "08:30:00" and "10:30:00" then 'mon_morning'
        when extract(dayofweek from date(job_dt_local)) = 2 and time(job_dt_local) between "10:30:00" and "13:30:00" then 'mon_lunch'
        when extract(dayofweek from date(job_dt_local)) = 2 and time(job_dt_local) between "13:30:00" and "16:30:00" then 'mon_noon'
        when extract(dayofweek from date(job_dt_local)) = 2 and time(job_dt_local) between "16:30:00" and "21:30:00" then 'mon_dinner'
        when extract(dayofweek from date(job_dt_local)) = 2 and time(job_dt_local) between "21:30:00" and "23:30:00" then 'mon_night'
    
        -- Tuesday
        when extract(dayofweek from date(job_dt_local)) = 3 and time(job_dt_local) between "08:30:00" and "10:30:00" then 'tues_morning'
        when extract(dayofweek from date(job_dt_local)) = 3 and time(job_dt_local) between "10:30:00" and "13:30:00" then 'tues_lunch'
        when extract(dayofweek from date(job_dt_local)) = 3 and time(job_dt_local) between "13:30:00" and "16:30:00" then 'tues_noon'
        when extract(dayofweek from date(job_dt_local)) = 3 and time(job_dt_local) between "16:30:00" and "21:30:00" then 'tues_dinner'
        when extract(dayofweek from date(job_dt_local)) = 3 and time(job_dt_local) between "21:30:00" and "23:30:00" then 'tues_night'
    
        -- Wednesday
        when extract(dayofweek from date(job_dt_local)) = 4 and time(job_dt_local) between "08:30:00" and "10:30:00" then 'wed_morning'
        when extract(dayofweek from date(job_dt_local)) = 4 and time(job_dt_local) between "10:30:00" and "13:30:00" then 'wed_lunch'
        when extract(dayofweek from date(job_dt_local)) = 4 and time(job_dt_local) between "13:30:00" and "16:30:00" then 'wed_noon'
        when extract(dayofweek from date(job_dt_local)) = 4 and time(job_dt_local) between "16:30:00" and "21:30:00" then 'wed_dinner'
        when extract(dayofweek from date(job_dt_local)) = 4 and time(job_dt_local) between "21:30:00" and "23:30:00" then 'wed_night'
        
        -- Thursday
        when extract(dayofweek from date(job_dt_local)) = 5 and time(job_dt_local) between "08:30:00" and "10:30:00" then 'thur_morning'
        when extract(dayofweek from date(job_dt_local)) = 5 and time(job_dt_local) between "10:30:00" and "13:30:00" then 'thur_lunch'
        when extract(dayofweek from date(job_dt_local)) = 5 and time(job_dt_local) between "13:30:00" and "16:30:00" then 'thur_noon'
        when extract(dayofweek from date(job_dt_local)) = 5 and time(job_dt_local) between "16:30:00" and "21:30:00" then 'thur_dinner'
        when extract(dayofweek from date(job_dt_local)) = 5 and time(job_dt_local) between "21:30:00" and "23:30:00" then 'thur_night'
        
        -- Friday
        when extract(dayofweek from date(job_dt_local)) = 6 and time(job_dt_local) between "08:30:00" and "10:30:00" then 'fri_morning'
        when extract(dayofweek from date(job_dt_local)) = 6 and time(job_dt_local) between "10:30:00" and "13:30:00" then 'fri_lunch'
        when extract(dayofweek from date(job_dt_local)) = 6 and time(job_dt_local) between "13:30:00" and "16:30:00" then 'fri_noon'
        when extract(dayofweek from date(job_dt_local)) = 6 and time(job_dt_local) between "16:30:00" and "21:30:00" then 'fri_dinner'
        when extract(dayofweek from date(job_dt_local)) = 6 and time(job_dt_local) between "21:30:00" and "23:30:00" then 'fri_night'
    
        -- Saturday
        when extract(dayofweek from date(job_dt_local)) = 7 and time(job_dt_local) between "08:30:00" and "10:30:00" then 'sat_morning'
        when extract(dayofweek from date(job_dt_local)) = 7 and time(job_dt_local) between "10:30:00" and "13:30:00" then 'sat_lunch'
        when extract(dayofweek from date(job_dt_local)) = 7 and time(job_dt_local) between "13:30:00" and "16:30:00" then 'sat_noon'
        when extract(dayofweek from date(job_dt_local)) = 7 and time(job_dt_local) between "16:30:00" and "21:30:00" then 'sat_dinner'
        when extract(dayofweek from date(job_dt_local)) = 7 and time(job_dt_local) between "21:30:00" and "23:30:00" then 'sat_night'
        
        -- Sunday
        when extract(dayofweek from date(job_dt_local)) = 1 and time(job_dt_local) between "08:30:00" and "10:30:00" then 'sun_morning'
        when extract(dayofweek from date(job_dt_local)) = 1 and time(job_dt_local) between "10:30:00" and "13:30:00" then 'sun_lunch'
        when extract(dayofweek from date(job_dt_local)) = 1 and time(job_dt_local) between "13:30:00" and "16:30:00" then 'sun_noon'
        when extract(dayofweek from date(job_dt_local)) = 1 and time(job_dt_local) between "16:30:00" and "21:30:00" then 'sun_dinner'
        when extract(dayofweek from date(job_dt_local)) = 1 and time(job_dt_local) between "21:30:00" and "23:30:00" then 'sun_night'
      end as tag
    , count(distinct job_uuid) as jobs
  
  from `postmates-reporting.stats.job` j

  where 1 = 1
    and complete is true
    and j.fsm_state in ('DidCompleteDropoff','DidAdminCancel','DidCustomerCancel','DidDispatchCancel','DidDispatchRequestException')
    and j.fraud_canceled is false
    and j.charge_state != 'auth_failed'
    and j.is_pickup is false
    and j.is_return_job is false
    
    -- look back 2 weeks (starting from last monday)
    and date(job_dt_local) < date_sub(date_trunc(current_date("America/Los_Angeles"), week(monday)), interval 1 week)
    and date(job_dt_local) >= date_sub(date_trunc(current_date("America/Los_Angeles"), week(monday)), interval 3 week)

  group by 1, 2, 3, 4
),

peak_deliveries as (

  select
      market
    , courier_uuid
    , tag
    , avg(jobs) jobs
  
  from peak_deliveries_s1
  
  group by 1, 2, 3

)

select distinct 
    market
  , tag
  , CONCAT(market, "_", tag) as concatenated
  , ceil(PERCENTILE_DISC(jobs, 0.85) OVER ( PARTITION BY market,tag )) AS PercentileDisc85

FROM peak_deliveries

where 1 = 1
  and tag is not null
  and market in ("Los Angeles", "Phoenix", "San Diego", "Long Beach", "Orange County", "San Bernardino", "San Francisco", "Portland")
order by 4 desc"""

peak = bqclient.query(peak).result().to_dataframe()

#Market ids.
markets="SELECT distinct id as market_id, name as market, id as market_id2 FROM `datafall-prod.postal_main.core_market`"
markets = bqclient.query(markets).result().to_dataframe()


In [5]:
#Location specific
def loc_spec(df):
  loc_specific = ['McD_BPD', 'SGF_NYC', 'CFA_BPD']
  uuids=[mcd, sgf, cfa]
  today=arrow.now().format('MMDDYY')
  for index, store in enumerate(loc_specific):

    temp=df[df['Name']==store]
    incentive=pd.DataFrame(columns=incentive_columns)
    incentive['place_uuids'],incentive['Market']=uuids[index]['place_uuids'],uuids[index]['Market']
    incentive['supports_flexible_enrollments']=False
    incentive['incentive_type']=list(temp['Incentive_type'])[0]
    incentive['name'],incentive['description']=list(temp['Name'])[0],list(temp['Name'])[0]
    incentive['payout']=int(temp['Payout'])
    incentive['enrollment_start_dt']=pd.to_datetime(list(temp['Enrollement start date'])[0])
    incentive['enrollment_end_dt']=pd.to_datetime(list(temp['Enrollement end date'])[0])
    incentive['payout_amount']=incentive['payout']*100  
    incentive['canvas_id']=today+'_canvas_'+ store
    incentive['variant_id']=today+'_variant_'+ store
    incentive=pd.merge(incentive, geo, left_on='Market', right_on='name', how='left')
    incentive['geo_region_id']=incentive['id']
    incentive['timezone_x']=incentive['timezone_y']
    incentive=incentive.iloc[:, : 15]
    incentive=incentive.rename(columns = {'name_x':'name', 'timezone_x':'timezone'})
    incentive=incentive.drop_duplicates()
    os.chdir(weekday_dir)
    incentive.to_csv(store+'.csv')

In [6]:
def mon_thurs(df, avg_earnings):
  avg_earnings=avg_earnings
  today=arrow.now().format('MMDDYY')
  temp=df[df['Name']=='mon_thurs']
  temp.columns=[x.lower() for x in temp.columns]
  incentive=pd.DataFrame(columns=incentive_columns)
  markets_mt=str(temp['market'][0])
  markets_mt=markets_mt.split(",")
  markets_mt=list(itertools.chain.from_iterable(itertools.repeat(x, 8) for x in markets_mt))
  incentive['market']=markets_mt
  incentive['incentive_type']='Guaranteed Earnings'
  incentive['name']='Peak'
  incentive['supports_flexible_enrollments']=False
  incentive['canvas_id']=today+'_canvas_'+ 'mon_thur_v1'
  incentive['variant_id']=today+'_variant_'+ 'mon_thur_v1'
  incentive=pd.merge(incentive, geo, left_on='market', right_on='name', how='left')
  incentive['geo_region_id']=incentive['id']
  incentive['timezone_x']=incentive['timezone_y']
  incentive=incentive.iloc[:, : 15]
  incentive=incentive.rename(columns = {'name_x':'name', 'timezone_x':'timezone'})
  incentive['payout_amount']=incentive['payout']*100

#Description

  description=['Monday_Lunch,Monday_Dinner,Tuesday_Lunch,Tuesday_Dinner,Wednesday_Lunch,Wednesday_Dinner,Thursday_Lunch,Thursday_Dinner']
  description=description*5
  description=','.join(description)
  description=description.split(',')
  incentive['description']=description


#Enrollment dates.
  today = datetime.date.today()
  next_mon= today + datetime.timedelta(days=-today.weekday(), weeks=1)
  next_tue= today + datetime.timedelta(days=1-today.weekday(), weeks=1)
  next_wed= today + datetime.timedelta(days=2-today.weekday(), weeks=1)
  next_thu= today + datetime.timedelta(days=3-today.weekday(), weeks=1)

  dates=[next_mon, next_tue,next_wed, next_thu]
  dates=list(itertools.chain.from_iterable(itertools.repeat(x, 2) for x in dates))
  dates=dates*5
  incentive['enrollment_start_dt']=dates
  incentive['enrollment_end_dt']=dates

#Goals and payouts
  incentive=incentive.merge(peak, on='market', how='left')
  incentive['goals']=incentive['PercentileDisc85']

  avg_earnings=pd.DataFrame(avg_earnings)
  avg_earnings.rename({'f0_': 'market'}, axis=1, inplace=True)
  incentive=incentive.merge(avg_earnings, on='market', how='left')
  incentive['payout']=incentive['goals']*incentive['avg_courier_price']

  incentive['payout']=incentive['payout'].round(decimals=0)
  incentive['payout_amount']=incentive['payout']*100

  today=arrow.now().format('MMDDYY')
  name=str(today)+'_mon_thurs_v1.csv'
  os.chdir(weekday_dir)
  incentive.to_csv(name)

In [7]:
#Starters
def starters(group):
  os.chdir('/content/drive/MyDrive')
  today=arrow.now().format('MMDDYY')
  starters=pd.read_csv('starters.csv')
  starters['canvas_id']=str(today)+'_canvas_Starters_Tier'+str(group)
  starters['variant_id']=str(today)+'_canvas_Starters_Tier'+str(group)
  today = datetime.date.today()
  next_fri= today + datetime.timedelta(days=4-today.weekday(), weeks=0)
  next_thu= today + datetime.timedelta(days=3-today.weekday(), weeks=1)
  starters['enrollment_start_dt']=next_fri
  starters['enrollment_end_dt']=next_thu
  dir='/content/drive/My Drive/Incentive QCs/'+today_path+'/weekends'
  os.chdir(dir)
  name=str(today)+'_Starters_Tier_'+str(group)+'_v1.csv'
  os.chdir(weekend_dir)
  starters.to_csv(name)

In [8]:
def peaks(df, peak, avg_earnings):
  today=arrow.now().format('MMDDYY')
  temp=df[df['Name']=='Peak']
  temp.columns=[x.lower() for x in temp.columns]
  incentive=pd.DataFrame(columns=incentive_columns)
  markets=str(temp['market'][2])
  markets=markets.split(",")
  markets_all=list(itertools.chain.from_iterable(itertools.repeat(x, 6) for x in markets))
  incentive['market']=markets_all
  incentive['incentive_type']='Guaranteed Earnings'
  incentive['name']='Peak'
  incentive['supports_flexible_enrollments']=False  
  incentive['canvas_id']=today+'_canvas_'+ 'Peak_v1'
  incentive['variant_id']=today+'_variant_'+ 'Peak_v1'
  incentive=pd.merge(incentive, geo, left_on='market', right_on='name', how='left')
  incentive['geo_region_id']=incentive['id']
  incentive['timezone_x']=incentive['timezone_y']
  incentive=incentive.iloc[:, : 15]
  incentive=incentive.rename(columns = {'name_x':'name', 'timezone_x':'timezone'})
  
  #Description
  description=['Friday_Lunch', 'Friday_Dinner', 'Saturday_Lunch', 'Saturday_Dinner', 'Sunday_Lunch', 'Sunday_Dinner']
  description=description*len(markets)
  description=','.join(description)
  description=description.split(',')
  incentive['description']=description

  #Enrollment dates.
  today = datetime.date.today()
  next_fri= today + datetime.timedelta(days=4-today.weekday(), weeks=0)
  next_sat= today + datetime.timedelta(days=5-today.weekday(), weeks=0)
  next_sun= today + datetime.timedelta(days=6-today.weekday(), weeks=0)

  dates=[next_fri, next_sat,next_sun]
  dates=list(itertools.chain.from_iterable(itertools.repeat(x, 2) for x in dates))
  dates=dates*len(markets)
  incentive['enrollment_start_dt']=dates
  incentive['enrollment_end_dt']=dates

  #Goals and payouts
  incentive=incentive.merge(peak, on='market', how='left')
  incentive['goals']=incentive['PercentileDisc85']

  avg_earnings=pd.DataFrame(avg_earnings)
  avg_earnings.rename({'f0_': 'market'}, axis=1, inplace=True)
  incentive=incentive.merge(avg_earnings, on='market', how='left')
  incentive['payout']=incentive['goals']*incentive['avg_courier_price']

  incentive['payout']=incentive['payout'].round(decimals=0)
  incentive['payout_amount']=incentive['payout']*100
  name=str(today)+'_Peak_v1.csv'
  os.chdir(weekday_dir)
  incentive.to_csv(name)

In [9]:
def react(df):
  os.chdir('/content/drive/MyDrive')
  temp=df[df['Name']=='Reactivation_BPDs']
  temp.columns=[x.lower() for x in temp.columns]
  incentive=pd.DataFrame(columns=incentive_columns)

  markets=str(temp['market'][3])
  markets=markets.split(",")
  markets_all=list(itertools.chain.from_iterable(itertools.repeat(x, 6) for x in markets))  

  #Markets, timezone, incentive type, canvas, and variant.
  incentive['market']=markets_all
  incentive['incentive_type']='Bonus Per Delivery'
  incentive['name']='Reactivation_BPDs'
  incentive['supports_flexible_enrollments']=False
  incentive['canvas_id']=today+'_canvas_'+ 'React_v1'
  incentive['variant_id']=today+'_variant_'+ 'React_v1'
  incentive=pd.merge(incentive, geo, left_on='market', right_on='name', how='left')
  incentive['geo_region_id']=incentive['id']
  incentive['timezone_x']=incentive['timezone_y']
  incentive=incentive.iloc[:, : 15]
  incentive=incentive.rename(columns = {'name_x':'name', 'timezone_x':'timezone'})

  #Description
  description=['Friday_Lunch', 'Friday_Dinner', 'Saturday_Lunch', 'Saturday_Dinner', 'Sunday_Lunch', 'Sunday_Dinner']
  description=description*len(markets)
  description=','.join(description)
  description=description.split(',')
  incentive['description']=description

  markets=str(temp['market'][3])
  markets=markets.split(",")
  markets_all=list(itertools.chain.from_iterable(itertools.repeat(x, 6) for x in markets))

  #Enrollement date
  incentive['enrollment_start_dt']=dates
  incentive['enrollment_end_dt']=dates

  #Payouts
  lunch=int(temp['payout'][3][0])
  dinner=int(temp['payout'][3][2])

  incentive.loc[incentive['description'].str.contains("Lunch"), 'payout'] = lunch
  incentive.loc[incentive['description'].str.contains("Dinner"), 'payout'] = dinner

  incentive['payout_amount']=incentive['payout']*100
  name=str(today)+'_react_v1'
  dir='/content/drive/My Drive/Incentive QCs/'+today_path+'/weekends'
  os.chdir(dir)
  os.chdir(weekday_dir)
  incentive.to_csv(name)  
